In [ ]:
!pip install dash

In [ ]:
!pip install plotly
!pip install httpx==0.20 dash plotly

In [ ]:
# importing relevant libraries

import dash
import pandas as pd
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output



In [ ]:
# load the airline data into pandas dataframe
airline_data = pd.read_csv('Dashboarding/airline_data.csv', 
                          encoding='ISO-8859-1',
                          dtype={'Div1Airport': str, 'Div1TailNum': str,
                                'Div2Airport': str, 'Div2TailNum': str})

#airline_data.head()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

In [ ]:
# Build dash layout
app.layout = html.Div(children=[ html.H1('Flight Delay Statisticts', style={'textAlign': 'center', 'color': '#503d36', 'font-size': 30}),
                               html.Div(['Input Year: ', dcc.Input(id='input-year',
                                                                 value='2010',
                                                                 type='number',
                                                                 style={'height': '35px', 'font-size': 30}),],
                                       style={'font-size': 30}),
                               html.Br(),
                               html.Br(),
                               # Segment 1
                               html.Div([html.Div(dcc.Input(id='carrier-plot')),
                                        html.Div(dcc.Input(id='weather-plot'))], style={'display': 'flex'}),
                               # Segment 2
                               html.Div([html.Div(dcc.Input(id='nas-plot')),
                                        html.Div(dcc.Input(id='security-plot'))], style={'display': 'flex'}),
                               # Segment 3
                               html.Div(html.Div(dcc.Input(id='late-plot'), style={'width': '65%'}))])


In [ ]:
# define a funtion for computation
''' compute_info function description
The function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments: 
        airline_data: Input airline data
        entered_year: Input year for which computation needs to be performed

Returns:
        computed average dataframes for carrier delay, weather delay, NAS delay, security delay and delays caused by late aircraft
'''
def compute_info(airline_data, entered_year):
    # Select the data
    df = airline_data[airline_data['Year']==int(entered_year)]
    
    # compute average delays for the various components
    avg_carrier_delay = df.groupby(['Month', 'Reporting Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather_delay = df.groupby(['Month', 'Reporting Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS_delay = df.groupby(['Month', 'Reporting Airline'])['NASDelay'].mean().reset_index()
    avg_security_delay = df.groupby(['Month', 'Reporting Airline'])['SecurityDelay'].mean().reset_index()
    avg_late_delay = df.groupby(['Month', 'Reporting Airline'])['LateAircraftDelay'].mean().reset_index()
    
    # return the computed averages
    return avg_carrier_delay, avg_weather_delay, avg_NAS_delay, avg_security_delay, avg_late_delay



In [ ]:
# Callback decorator
app.callback([Output(component_id='carrier-fig', component_property='figure'),
              Output(component_id='weather-fig', component_property='figure'),
              Output(component_id='nas-fig', component_property='figure'),
              Output(component_id='security-fig', component_property='figure'),
              Output(component_id='late-fig', component_property='figure')],
            Input(component_id='input-year', component_property='value'))

In [ ]:
# Computation of the callback function and return graph
def get_graph(entered_year):
    
    # compute the required information for generating the graph
    avg_carrier_delay, avg_weather_delay, avg_NAS_delay, avg_security_delay, avg_late_delay = compute_info(airline_data, entered_year)
    
    # generate the line graph for the various delays
    carrier_fig=px.line(avg_carrier_delay, x='Month', y='CarrierDelays', color='Reporting Airline', title='Average Carrier delays (minutes) by airline')
    weather_fig=px.line(avg_weather_delay, x='Month', y='WeatherDelays', color='Reporting Airline', title='Average Weather delays (minutes) by airline')
    nas_fig=px.line(avg_NAS_delay, x='Month', y='NASDelays', color='Reporting Airline', title='Average NAS delays (minutes) by airline')
    security_fig=px.line(avg_security_delay, x='Month', y='SecurityDelays', color='Reporting Airline', title='Average Security delays (minutes) by airline')
    late_fig=px.line(avg_late_delay, x='Month', y='LateAircraftDelays', color='Reporting Airline', title='Average Late Aircraft delays (minutes) by airline')
    
    # return the plots
    return [carrier-fig, weather-fig, nas-fig, security-fig, late-fig]

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()